In [17]:
from google.colab import drive
drive.mount('/content/drive')
#!ls "/content/drive/My Drive/collab_sandbox"
%cd drive/MyDrive/collab_sandbox/text_generation/finetune_gpt2/
!ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[Errno 2] No such file or directory: 'drive/MyDrive/collab_sandbox/text_generation/finetune_gpt2/'
/content/drive/MyDrive/collab_sandbox/text_generation/finetune_gpt2
 big_tensor.pt
 Conditional_Text_Generation_with_GPT_2.ipynb
 finetune_gpt_2_pure_sentences.ipynb
'GPT_2_Fine_Tuning_w_Hugging_Face_&_PyTorch.ipynb'
 gpt2_train
 gpt2_train_5_2_21
 gpt2_train_6_17_21_clean_sentences
 gpt2_train_6_17_21_clean_sentences_2
 gpt2_train_6_17_21_dirty_sentences
 gpt2_train_6_17_21_dirty_sentences_2
 label_sentences__4_25_21_1000.csv
 label_sentences__4_26_21_1000.csv
 medium_sentenses_labeled_all__4_23_21_80479.csv
 medium_sentenses_labeled_all__4_23_21_80479.gsheet
 medium_sentenses_labeled_all__4_23_21_80479.pt
'medium_sentenses_labeled_all__4_23_21_80479__Sun May  2 01_18_31 2021_1619918311.231216.csv'
'medium_sentenses_labeled_all__4_23_21_80479__Sun May  2 01_18_

Related article: https://www.ivanlai.project-ds.net/post/conditional-text-generation-by-fine-tuning-gpt-2

Preprocessing code in [this](https://github.com/ivanlai/Conditional_Text_Generation) Github repository.

### Install and import libraries

In [18]:
%%time
%%capture
!pip install transformers

CPU times: user 11.3 ms, sys: 62.4 ms, total: 73.7 ms
Wall time: 2.57 s


Check GPU memory available (Colab could offer 12GB or 16GB). 

Our configuration works on 16GB. The batch size needs to be reduced if only 12GB were available.




In [ ]:
!nvidia-smi

Wed Jun 16 22:49:17 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    24W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [21]:
import os
import io
import requests
import numpy as np
import pandas as pd
import re
import zipfile
import random
import time
import csv
import datetime
from sklearn.model_selection import train_test_split
from itertools import compress
from collections import Counter, defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from transformers import AutoTokenizer, AutoConfig, AutoModelForPreTraining, \
                         AdamW, get_linear_schedule_with_warmup, \
                         TrainingArguments, BeamScorer, Trainer

import torch
from torch.utils.data import Dataset, random_split, DataLoader, \
                             RandomSampler, SequentialSampler

from IPython.display import clear_output
from tqdm.notebook import tqdm

print(f"PyTorch version: {torch.__version__}")

PyTorch version: 1.8.1+cu101


### Configurations

In [22]:
# DEBUG           = False

# INPUT_DIR       = 'articles'

USE_APEX        = True
APEX_OPT_LEVEL  = 'O1'

MODEL           = 'gpt2' #{gpt2, gpt2-medium, gpt2-large, gpt2-xl}

UNFREEZE_LAST_N = 6 #The last N layers to unfreeze for training

SPECIAL_TOKENS  = { "bos_token": "<|BOS|>",
                    "eos_token": "<|EOS|>",
                    "unk_token": "<|UNK|>",                    
                    "pad_token": "<|PAD|>",
                    "sep_token": "<|SEP|>"}
                    
MAXLEN          = 256  #{768, 1024, 1280, 1600}

TRAIN_SIZE      = 0.8

if USE_APEX:
    TRAIN_BATCHSIZE = 4
    BATCH_UPDATE    = 16
else:
    TRAIN_BATCHSIZE = 2
    BATCH_UPDATE    = 32

EPOCHS          = 4
LR              = 5e-4
EPS             = 1e-8
WARMUP_STEPS    = 1e2

SEED            = 2020

In [23]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(SEED)

### Datasets and loaders

In [6]:
# MEDIUM_DATASET = pd.read_csv("medium_sentenses_labeled_all__4_23_21_80479.csv", encoding='utf-8')
MEDIUM_DATASET = pd.read_csv("/content/drive/My Drive/collab_sandbox/NER/ner_tenses_recognition/datasets/comb_med_2119_6_15_21/comb_med_2119_6_17_21_sentences_labeled__Thu Jun 17 13:02:27 2021_1623934947.656283.csv", encoding='utf-8')
MEDIUM_DATASET

,sentence,topic,tenses
0,"It pollutes the environment, it's limited in s...",science,a1_present_simple_3d_pers a1_to_be_present_is_...
1,"Of course, nowadays electric cars are undoubte...",science,a1_to_be_present_is_am_are a1_comparative_long...
2,"Yet, it's worth talking about biofuels, as the...",science,a1_to_be_present_is_am_are a1_present_simple_r...
3,"While there are many types of biofuels, we'll ...",science,a1_there_is_am_are a1_future_simple a1_superla...
4,"There are also biofuels for planes and boats, ...",science,a1_there_is_am_are
...,...,...,...
88691,"She's bright, bubbly, and full of the future t...",travel,a1_to_be_present_is_am_are a1_present_simple_3...
88692,"As I wrote in my book, ""Offshoring The Middle ...",travel,a1_past_simple_irreg a1_present_simple_reg_act
88693,"In fact, I'd argue it's our duty to encourage ...",travel,a1_to_be_present_is_am_are
88694,"It's not easy, but not impossible; in fact, no...",travel,a1_to_be_present_is_am_are a1_present_simple_r...


In [ ]:
len(MEDIUM_DATASET[MEDIUM_DATASET['tenses'].str.contains('a1_past_simple_irreg')]) 

7648

### Loading Tokenizer, Config and Model

In [7]:
def get_tokenier(special_tokens=None):
    tokenizer = AutoTokenizer.from_pretrained(MODEL) #GPT2Tokenizer

    if special_tokens:
        tokenizer.add_special_tokens(special_tokens)
        print("Special tokens added")
    return tokenizer

def get_model(tokenizer, special_tokens=None, load_model_path=None):

    #GPT2LMHeadModel
    if special_tokens:
        config = AutoConfig.from_pretrained(MODEL, 
                                            bos_token_id=tokenizer.bos_token_id,
                                            eos_token_id=tokenizer.eos_token_id,
                                            sep_token_id=tokenizer.sep_token_id,
                                            pad_token_id=tokenizer.pad_token_id,
                                            output_hidden_states=False)
    else: 
        config = AutoConfig.from_pretrained(MODEL,                                     
                                            pad_token_id=tokenizer.eos_token_id,
                                            output_hidden_states=False)    

    #----------------------------------------------------------------#
    model = AutoModelForPreTraining.from_pretrained(MODEL, config=config)

    if special_tokens:
        #Special tokens added, model needs to be resized accordingly
        model.resize_token_embeddings(len(tokenizer))

    if load_model_path:
        model.load_state_dict(torch.load(load_model_path))

    model.cuda()
    return model

In [39]:
%%time

tokenizer = get_tokenier(special_tokens=SPECIAL_TOKENS)
model = get_model(tokenizer, 
                  special_tokens=SPECIAL_TOKENS,
                #   load_model_path='pytorch_model.bin'
                 )

Special tokens added
CPU times: user 2.68 s, sys: 413 ms, total: 3.09 s
Wall time: 5.37 s


In [10]:
import pandas as pd
import random
import spacy

# Load English tokenizer, tagger, parser and NER
nlp = spacy.load("en_core_web_sm")

class myDataset(Dataset):

  def __init__(self, dataset, tokenizer=tokenizer, randomize=True):
    self.tokenizer = tokenizer 
    self.dataset = dataset.reset_index()
    self.randomize = randomize

  def __len__(self):
    return len(self.dataset)

  def __getitem__(self, idx):
      keywords = self.dataset['tenses'][idx].split()
      if self.randomize:
        random.shuffle(keywords)
      keywords = " ".join(keywords)
      
      topic = self.dataset['topic'][idx]
      sentence = self.dataset['sentence'][idx]
      # first_POS = self.dataset['first_pos'][idx]
      
      input = SPECIAL_TOKENS['bos_token'] + topic + \
              SPECIAL_TOKENS['sep_token'] + keywords + SPECIAL_TOKENS['sep_token'] + \
              sentence + SPECIAL_TOKENS['eos_token']
              # SPECIAL_TOKENS['sep_token'] + first_POS + SPECIAL_TOKENS['sep_token'] + \

      encodings_dict = tokenizer(input,                                   
                                  truncation=True, 
                                  max_length=256, 
                                  padding="max_length")   
      
      input_ids = encodings_dict['input_ids']
      attention_mask = encodings_dict['attention_mask']
      
      return {'label': torch.tensor(input_ids, device='cuda'),
              'input_ids': torch.tensor(input_ids, device='cuda'), 
              'attention_mask': torch.tensor(attention_mask, device='cuda')}

In [13]:
# - Freeze selective layers:
# - Freeze all layers except last n:
for parameter in model.parameters():
    parameter.requires_grad = False

for i, m in enumerate(model.transformer.h):        
    #Only un-freeze the last n transformer blocks
    if i+1 > 12 - UNFREEZE_LAST_N:
        for parameter in m.parameters():
            parameter.requires_grad = True 

for parameter in model.transformer.ln_f.parameters():        
    parameter.requires_grad = True

for parameter in model.lm_head.parameters():        
    parameter.requires_grad = True

In [14]:
train_data, val_data = train_test_split(MEDIUM_DATASET, test_size=0.12)

train_dataset = myDataset(train_data)
val_dataset = myDataset(val_data, randomize=False)

f'train={len(train_dataset) :} val={len(val_dataset) :}'
# next(iter(train_dataset))

'train=78052 val=10644'

In [15]:
!pip install -q wandb
%env WANDB_PROJECT=grammar_contructions_generation
import wandb
wandb.login()

     |████████████████████████████████| 1.8MB 7.8MB/s 
     |████████████████████████████████| 133kB 54.9MB/s 
     |████████████████████████████████| 102kB 13.0MB/s 
     |████████████████████████████████| 163kB 58.2MB/s 
     |████████████████████████████████| 71kB 11.0MB/s 
env: WANDB_PROJECT=grammar_contructions_generation


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

### Fine-tune GPT2 using Trainer

In [16]:
training_args = TrainingArguments(
    output_dir="./gpt2_train_6_17_21_clean_sentences_2/",
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    gradient_accumulation_steps=BATCH_UPDATE,
    evaluation_strategy="steps",
    fp16=True,
    fp16_opt_level=APEX_OPT_LEVEL,
    warmup_steps=WARMUP_STEPS,    
    learning_rate=LR,
    adam_epsilon=EPS,
    weight_decay=0.01,        
    save_total_limit=1,
    load_best_model_at_end=True,
    eval_steps=100,
    logging_steps=100,
    report_to="wandb",
    metric_for_best_model='eval_loss',
    greater_is_better=False
)


trainer = Trainer(
    model=model,
    args=training_args,    
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer
)


trainer.train()
trainer.save_model()  
wandb.finish()   

wandb: Currently logged in as: dimweb (use `wandb login --relogin` to force relogin)


Step,Training Loss,Validation Loss
100,2.222100,0.403780
200,0.419400,0.390521
300,0.404700,0.385374
400,0.389500,0.383817
500,0.381700,0.381682
600,0.380600,0.380377
700,0.367600,0.381590
800,0.362800,0.380273
900,0.363200,0.379537
1000,0.354600,0.381365


train/loss,0.351
train/learning_rate,1e-05
train/epoch,4.0
train/global_step,1216
_runtime,12644
_timestamp,1623949233
_step,24
eval/loss,0.38056
eval/runtime,140.1014
eval/samples_per_second,75.974
train/train_runtime,12644.1257


train/loss,█▁▁▁▁▁▁▁▁▁▁▁
train/learning_rate,██▇▆▆▅▄▄▃▂▂▁
train/epoch,▁▁▂▂▂▂▃▃▃▃▄▄▅▅▅▅▆▆▇▇▇▇███
train/global_step,▁▁▂▂▂▂▃▃▄▄▄▄▅▅▅▅▆▆▇▇▇▇███
_runtime,▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▇▇▇▇███
_timestamp,▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▇▇▇▇███
_step,▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇▇██
eval/loss,█▄▃▂▂▁▂▁▁▂▁▁
eval/runtime,▁▅▆▆▅▅▅▆▅▆██
eval/samples_per_second,█▄▃▃▄▄▄▃▄▃▁▁
train/train_runtime,▁


In [ ]:
trainer.save_model()  

In [ ]:
# Save to G-Drive ----------------------------------#
# !cp -r 'pytorch_model.bin' '/content/drive/MyDrive/Colab Notebooks/Text Generation/pytorch_model_V2.bin'


In [ ]:
!ls

 Conditional_Text_Generation_with_GPT_2.ipynb
'GPT_2_Fine_Tuning_w_Hugging_Face_&_PyTorch.ipynb'
 gpt2_train
 medium_sentenses_labeled_all__4_23_21_80479.csv
 medium_sentenses_labeled_all__4_23_21_80511.csv
 PPLM.ipynb
 runs
 Untitled0.ipynb


### Generating text with Fine-tuned GPT-2 model

In [ ]:
# !cp -r '/content/drive/MyDrive/Colab Notebooks/Text Generation/pytorch_model_V2.bin' 'pytorch_model.bin' 

In [40]:

model = get_model(tokenizer, 
                  special_tokens=SPECIAL_TOKENS,
                  load_model_path='./gpt2_train_6_17_21_clean_sentences_2/pytorch_model.bin')

# model_old = get_model(tokenizer, 
#                   special_tokens=SPECIAL_TOKENS,
#                   load_model_path='./gpt2_train/gpt_tenses_baseline.bin')

In [ ]:
tokenizer = get_tokenier(special_tokens=SPECIAL_TOKENS)

In [41]:
def generate_text_new_model(model_some):
  text = "He"
  # topic = 'artificial-intelligence'
  # tenses = "a1_can"
  # first_POS = 'PRP'
  topic = 'startups'
  tenses = "a1_to_be_present_is_am_are"

  # prompt = SPECIAL_TOKENS['bos_token'] + topic + \
  #         SPECIAL_TOKENS['sep_token'] + tenses + SPECIAL_TOKENS['sep_token'] + \
  #         SPECIAL_TOKENS['sep_token'] + first_POS + SPECIAL_TOKENS['sep_token'] + \
  #         text
  prompt = SPECIAL_TOKENS['bos_token'] + topic + \
          SPECIAL_TOKENS['sep_token'] + tenses + SPECIAL_TOKENS['sep_token'] + \
          text
          
  generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
  device = torch.device("cuda")
  generated = generated.to(device)

  model_some.eval();

  # Top-p (nucleus) text generation (10 samples):
  sample_outputs = model_some.generate(generated, 
                                  do_sample=True,   
                                  min_length=50, 
                                  max_length=256,
                                  top_k=30,                                 
                                  top_p=0.7,        
                                  temperature=0.9,
                                  repetition_penalty=2.0,
                                  num_return_sequences=10
                                  )
  print(prompt)
  for i, sample_output in enumerate(sample_outputs):
      text = tokenizer.decode(sample_output, skip_special_tokens=True)
      # a = len(topic) + len(tenses) + len(first_POS)
      a = len(topic) + len(tenses)
      print("{}: {}\n\n".format(i+1,  text[a:]))
      # print(text)

In [42]:
# generate_text_new_model(model)
# print("--------------------------------------")
generate_text_new_model(model)

<|BOS|>startups<|SEP|>a1_to_be_present_is_am_are<|SEP|>He
1: He is a founder and CEO of The Powerpuff Girls.com, an online dating platform where he's been featured on multiple national TV shows including Dancing with the Stars, The Ellen DeGeneres Show (where she was once nominated for her first ever Miss America), Love Actually, Housewives Of New Jersey as well - and


2: He is passionate about his business and wants to make a difference for the city.


3: He is a founder of the fund, where he invests in venture capital companies.


4: He is passionate about the idea of entrepreneurship and investing in startups.


5: He is passionate about the development of AI and robotics, which are emerging fields.


6: He is a former member of the board at Sequoia Capital and co-founder with Scott Goldstein.


7: He is a big believer in the power of entrepreneurship.


8: He is passionate about his passion for learning and creating.


9: He is passionate about the future of AI. "


10: He's also 

# Measure perplexity


In [34]:
!pip install -q nlp

     |████████████████████████████████| 1.7MB 5.3MB/s 
     |████████████████████████████████| 245kB 49.1MB/s 


In [35]:
from transformers import GPT2LMHeadModel, GPT2TokenizerFast
from nlp import load_dataset

device = 'cuda'
model_id = 'gpt2'
model = GPT2LMHeadModel.from_pretrained(model_id).to(device)
tokenizer = GPT2TokenizerFast.from_pretrained(model_id)

test = load_dataset('wikitext', 'wikitext-2-raw-v1', split='test')
encodings = tokenizer('\n\n'.join(test['text']), return_tensors='pt')

max_length = model.config.n_positions
stride = 512

lls = []
for i in tqdm(range(0, encodings.input_ids.size(1), stride)):
    begin_loc = max(i + stride - max_length, 0)
    end_loc = min(i + stride, encodings.input_ids.size(1))
    trg_len = end_loc - i    # may be different from stride on last loop
    input_ids = encodings.input_ids[:,begin_loc:end_loc].to(device)
    target_ids = input_ids.clone()
    target_ids[:,:-trg_len] = -100

    with torch.no_grad():
        outputs = model(input_ids, labels=target_ids)
        log_likelihood = outputs[0] * trg_len

    lls.append(log_likelihood)

ppl = torch.exp(torch.stack(lls).sum() / end_loc)

print(ppl)

Dataset wikitext downloaded and prepared to /root/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/8e456126357b4411737ead54576f99321fc077a0d4b64e4a724ab3454ba5b730. Subsequent calls will reuse this data.


Token indices sequence length is longer than the specified maximum sequence length for this model (287644 > 1024). Running this sequence through the model will result in indexing errors



tensor(25.1705, device='cuda:0')


In [ ]:
# Beam-search text generation:
sample_outputs = model.generate(generated, 
                                do_sample=True,   
                                max_length=MAXLEN,                                                      
                                num_beams=5,
                                repetition_penalty=5.0,
                                early_stopping=True,      
                                num_return_sequences=1
                                )

for i, sample_output in enumerate(sample_outputs):
    text = tokenizer.decode(sample_output, skip_special_tokens=True)
    a = len(title) + len(tenses)    
    print("{}: {}\n\n".format(i+1,  text[a:]))

1: The world’s most popular dating app crashed on its iOS and Android operating systems this week.

It has since been taken down from Apple’s App Store after users complained that the picture was too close to their faces. The company said in a statement: “This is not an isolated incident. We take great pride in what we do and look forward to working with others to make the sharing experience even better.”

Scroll down for video



Our hearts go out to all those who have lost their lives trying to find love over the past few days - but sadly it looks like there may be more to this tragic story than meets the eye

No-one knows exactly how many people were affected by the crash, which took place between late February and early March last year (stock image)

HOW IT WORKS A simple message will appear on your iPhone or iPad screen telling you if you are having trouble finding someone online. It then lets you meet up with them so they can pick you up where you left off. In other words, as soo

### Generating text with raw GPT2

In [ ]:
tokenizer = get_tokenier()
model = get_model(tokenizer)

In [ ]:
prompt = title

generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
device = torch.device("cuda")
generated = generated.to(device)

model.eval()
sample_outputs = model.generate(generated, 
                                do_sample=True,   
                                max_length=MAXLEN,                                                      
                                num_beams=5,
                                repetition_penalty=5.0,
                                early_stopping=True,      
                                num_return_sequences=1
                                )

for i, sample_output in enumerate(sample_outputs):
    print("{}: {}\n\n".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

0: We got a lot of grief when our photo became a meme.

"I didn't know what to do with it," she said. "It was just so much fun."




# Cluster Sentences

In [ ]:
import pandas as pd
MEDIUM_DATASET = pd.read_csv("medium_sentenses_labeled_all__4_23_21_80479.csv", encoding='utf-8')
MEDIUM_DATASET

,sentence,topic,tenses
0,"It pollutes the environment, it's limited in s...",science,a1_present_simple_3d_pers a1_to_be_present_is_...
1,"Of course, nowadays electric cars are undoubte...",science,a1_to_be_present_is_am_are a1_comparative_long...
2,"Yet, it's worth talking about biofuels, as the...",science,a1_to_be_present_is_am_are a1_present_simple_r...
3,"there are many types of biofuels, we'll focus ...",science,a1_there_is_am_are a1_future_simple a1_superla...
4,"There are also biofuels for planes and boats, ...",science,a1_there_is_am_are
...,...,...,...
80474,Thirty-seven agreed to join the quorum and tak...,business,a1_past_simple_reg a1_possessive_s_plurar
80475,"When the day arrived, the quorum moved.",business,a1_past_simple_reg a1_past_simple_reg
80476,"Firms that moved saw favorable press in CNN, R...",business,a1_past_simple_reg
80477,The movement has grown from 37 firms to over 1...,business,a1_past_simple_irreg


In [ ]:
# !pip install -U sentence-transformers

In [ ]:
# from sentence_transformers import SentenceTransformer
# sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')

from transformers import AutoTokenizer, AutoModel
import torch


#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask



#Sentences we want sentence embeddings for
sentences = ['This framework generates embeddings for each input sentence',
             'Sentences are passed as a list of string.',
             'The quick brown fox jumps over the lazy dog.']

#Load AutoModel from huggingface model repository

tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/bert-base-nli-mean-tokens")
sbert_model = AutoModel.from_pretrained("sentence-transformers/bert-base-nli-mean-tokens")
sbert_model = sbert_model.to(torch.device("cuda"))
def get_sentence_embeddings(sentences):
  #Tokenize sentences
  encoded_input = tokenizer(sentences, padding=True, truncation=True, max_length=256, return_tensors='pt')
  # это очень не эффективно сначала получать тензоры в CPU а потом 
  # переводить их на GPU вместо того чтобы сразу портировать их туда
  # https://towardsdatascience.com/7-tips-for-squeezing-maximum-performance-from-pytorch-ca4a40951259
  encoded_input = encoded_input.to(torch.device("cuda"))
  with torch.no_grad():
      model_output = sbert_model(**encoded_input)
  sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

  return sentence_embeddings

In [ ]:
get_sentence_embeddings(sentences)

tensor([[-0.1041,  0.5275,  1.1798,  ..., -0.4339, -0.6945,  0.5387],
        [-0.1312, -0.1739,  1.1052,  ...,  0.0262, -0.0027,  0.9161],
        [-0.7490,  0.7189, -1.0395,  ...,  0.1558,  1.0203,  0.0979]],
       device='cuda:0')

In [ ]:
from concurrent.futures import ProcessPoolExecutor
from tqdm.notebook import tqdm

def sentence_dataset_to_vectors(dataset=None, saving_file=None):
  sentence_vectors = []
  data_chunk = 1000
  data_parts = len(dataset) // data_chunk
  full_parts = data_chunk * data_parts

  for step in tqdm(range(data_parts)):
    sents = [str(item) for item in dataset[step*data_chunk:(step+1)*data_chunk]['sentence'].to_list()]
    sents_embeddings = get_sentence_embeddings(sents)
    sentence_vectors.append(sents_embeddings)
    del sents_embeddings
    torch.cuda.empty_cache()

  sents = [str(item) for item in dataset[full_parts:]['sentence'].to_list()]
  sents_embeddings = get_sentence_embeddings(sents)
  sentence_vectors.append(sents_embeddings)
  del sents_embeddings
  torch.cuda.empty_cache()

  sentence_vectors = torch.cat(sentence_vectors, dim=0)
  torch.save(sentence_vectors, saving_file)



In [ ]:
# sentence_dataset__to_vectors(dataset=MEDIUM_DATASET, saving_file='medium_sentenses_labeled_all__4_23_21_80479.pt')

In [ ]:
MEDIUM_EMBED = torch.tensor(torch.load('medium_sentenses_labeled_all__4_23_21_80479.pt').cuda(), dtype=torch.float16)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.


# fast clustering

In [ ]:
import torch
from tqdm.notebook import tqdm
A = torch.rand((100, 1000), dtype=torch.float16).cuda()

def effective_cosine_sim(matrix):
  """
  1 - u * v / |u|2 * |v|2
  """
  matrix_len = len(matrix)
  norms = torch.zeros((matrix_len), device='cuda', dtype=torch.float16)
  for i in range(matrix_len):
    norms[i] = torch.norm(matrix[i], p=2, dim=0)

  cosine_sim_matrix = torch.zeros((matrix_len, matrix_len), dtype=torch.float16, device='cuda')
  matrix_T = matrix.T
  for i in range(matrix_len):
    step_norms = norms[i] * norms
    need_mul = torch.matmul(matrix[i], matrix_T) / step_norms

    cosine_sim_matrix[i] = need_mul
    
  return cosine_sim_matrix

%time effective_cosine_sim(A)

CPU times: user 6.92 ms, sys: 0 ns, total: 6.92 ms
Wall time: 6.93 ms


tensor([[1.0000, 0.7393, 0.7461,  ..., 0.7437, 0.7622, 0.7559],
        [0.7393, 1.0000, 0.7544,  ..., 0.7432, 0.7529, 0.7520],
        [0.7461, 0.7544, 1.0000,  ..., 0.7476, 0.7617, 0.7524],
        ...,
        [0.7437, 0.7432, 0.7476,  ..., 1.0000, 0.7568, 0.7451],
        [0.7622, 0.7529, 0.7617,  ..., 0.7568, 1.0010, 0.7632],
        [0.7559, 0.7520, 0.7524,  ..., 0.7451, 0.7632, 1.0000]],
       device='cuda:0', dtype=torch.float16)

In [ ]:
%time effective_cosine_sim(MEDIUM_EMBED)

CPU times: user 15.9 s, sys: 12 s, total: 27.9 s
Wall time: 27.9 s


RuntimeError: ignored

In [ ]:
38400/175

219.42857142857142

In [ ]:
%timeit torch.rand((100, 1000), dtype=torch.float16, device='cuda')

The slowest run took 26.62 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 5: 12.1 µs per loop


In [ ]:
%timeit torch.rand((100, 1000), dtype=torch.float16,).cuda()

100 loops, best of 5: 2.48 ms per loop
